In [1]:
import pandas as pd
import numpy as np
import shapely
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, Dropout, UpSampling2D, Reshape
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [2]:
df = pd.read_pickle('./final.pkl')

In [3]:
### reference temperatures mapped
mb_am_temp = 56
mb_af_temp = 63
mb_pm_temp = 60

# Function to map AMafPM to ref_temp
def map_to_ref_temp(row):
    if row['AMafPM'] == [1,0,0]:
        return 56
    elif row['AMafPM'] == [0,1,0]:
        return 63
    elif row['AMafPM'] == [0,0,1]:
        return 60
    else:
        return None  # or a default value

# Apply the function to create the new column
df['ref_temp'] = df.apply(map_to_ref_temp, axis=1)

In [4]:
### make unique cols from feature grids
df['landuse']=df['feature_grids'].apply(lambda x: x['landuse'])
df['water']=df['feature_grids'].apply(lambda x: x['water'])
df['natural']=df['feature_grids'].apply(lambda x: x['natural'])
df['highway']=df['feature_grids'].apply(lambda x: x['highway'])

In [5]:
def rotate_and_augment(df):
    # Creating an empty DataFrame to store augmented data
    augmented_data = pd.DataFrame()
    
    # Iterating through each row in the original DataFrame
    for _, row in df.iterrows():
        # Storing original and rotated versions of each grid in a list
        all_versions = []
        
        # Original data
        original_data = {
            "Subgrid": row["Subgrid"],
            "height_grid": row["height_grid"],
            "landuse": row["landuse"],
            "water": row["water"],
            "natural": row["natural"],
            "highway": row["highway"],
            # ... include other non-rotated columns as needed
            "AMafPM": row["AMafPM"],
            "ref_temp": row["ref_temp"]
        }
        all_versions.append(original_data)
        
        # Creating rotated versions
        for angle in [90, 180, 270]:
            rotated_data = {
                "Subgrid": np.rot90(row["Subgrid"], k=angle//90),
                "height_grid": np.rot90(row["height_grid"], k=angle//90),
                "landuse": np.rot90(row["landuse"], k=angle//90, axes=(0, 1)),
                "water": np.rot90(row["water"], k=angle//90, axes=(0, 1)),
                "natural": np.rot90(row["natural"], k=angle//90, axes=(0, 1)),
                "highway": np.rot90(row["highway"], k=angle//90, axes=(0, 1)),
                # ... include other non-rotated columns as needed
                "AMafPM": row["AMafPM"],
                "ref_temp": row["ref_temp"]
            }
            all_versions.append(rotated_data)
        
        # Appending all versions to the augmented_data DataFrame
        augmented_data = augmented_data.append(all_versions, ignore_index=True)
    
    return augmented_data
df = rotate_and_augment(df)

/tmp/ipykernel_7849/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_7849/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_7849/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_7849/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_7849/

/tmp/ipykernel_7849/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_7849/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_7849/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_7849/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_7849/

In [6]:
df.to_pickle("./final_pro.pkl")